In [8]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("LOOOL").getOrCreate()

df = spark.read.option("header", True).csv("../00_data/transactions_train.csv")
df.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [12]:
df1 = df.show()

+----------+--------------------+----------+--------------------+----------------+
|     t_dat|         customer_id|article_id|               price|sales_channel_id|
+----------+--------------------+----------+--------------------+----------------+
|2018-09-20|000058a12d5b43e67...|0663713001|0.050830508474576264|               2|
|2018-09-20|000058a12d5b43e67...|0541518023| 0.03049152542372881|               2|
|2018-09-20|00007d2de826758b6...|0505221004| 0.01523728813559322|               2|
|2018-09-20|00007d2de826758b6...|0685687003|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0685687004|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0685687001|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0505221001|0.020322033898305086|               2|
|2018-09-20|00083cda041544b2f...|0688873012| 0.03049152542372881|               1|
|2018-09-20|00083cda041544b2f...|0501323011|0.053372881355932204|               1|
|201

In [14]:
rdd = spark.sparkContext.parallelize(df)

TypeError: cannot pickle '_thread.RLock' object